# Competitive Interaction on JudgeBench with Position Swapping

## Imports

In [162]:
import pandas as pd
from datasets import load_dataset
import os
from dotenv import load_dotenv
from autogen import ConversableAgent, GroupChat, GroupChatManager
import re
from tqdm import tqdm

## Data

In [163]:
JudgeBench_Claude = load_dataset("ScalerLab/JudgeBench", split="claude")

df = pd.DataFrame(JudgeBench_Claude)

df_sampled = df.iloc[:270].sample(n=100, random_state=42).reset_index(drop=True)

df_final = df_sampled[["question", "response_A", "response_B", "label"]]

print(df_final.info())
print(df_final.head(1))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   question    100 non-null    object
 1   response_A  100 non-null    object
 2   response_B  100 non-null    object
 3   label       100 non-null    object
dtypes: object(4)
memory usage: 3.3+ KB
None
                                            question  \
0  Under standard temperature and pressure condit...   

                                          response_A  \
0  Let's approach this step-by-step:\n\n1) The ra...   

                                          response_B label  
0  Let's approach this step-by-step:\n\n1) The ra...   B>A  


## Config

In [164]:
load_dotenv()

api_key = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_DEPLOYMENT_NAME")
api_version = os.getenv("AZURE_API_VERSION", "2023-12-01-preview")

config_list = [
    {
        "model": deployment_name,
        "api_key": api_key,
        "base_url": f"{endpoint}/openai/deployments/{deployment_name}/chat/completions?api-version={api_version}",
        "api_type": "azure",
        "api_version": api_version,  
        "temperature": 0,
        "cache_seed": 42
    }
]

llm_config={"config_list": config_list}

## System Design

In [165]:
agent_1_system_message =f"""
You are a Debate-Agent who's task is to support Response A as the more facutal correct response over Response B.
Argue convincingly and with clear reasoning why Response A is the more facutal correct and Response B is worse to persuade the Decision-Agent.
Use a structured format and about 200 words to present your arguments.
Always begin your response with: "As an independent evaluator I think ..."
Always end your ouput with: "Therefore Response A is the more factual correct response: {{"response": A}}
"""

agent_2_system_message =f"""
You are a Debate-Agent who's task is to support Response B as the more facutal correct response over Response A.
Argue convincingly and with clear reasoning why Response B is the more facutal correct and Response A is worse to persuade the Decision-Agent.
Use a structured format and about 200 words to present your arguments.
Always begin your response with: "As an independent evaluator I think ..."
Always end your ouput with: "Therefore Response B is the more factual correct response: {{"response": B}}
"""

agent_3_system_message ="""
You are the Decision-Agent in open debate setting.
Your task is to select the more facutal correct response between Response A and Response B based on the previous analyses by the Debate-Agents.
Give an explanation for your final conclusion.
Always end your output with stating if Response A or B is more factual correct by using a JSON-object with the follwing format: {{"response": A/B}}
"""

In [ ]:
agent_1 = ConversableAgent(
    name="Debate-Agent-A",
    llm_config=llm_config,
    system_message=agent_1_system_message,
    human_input_mode="NEVER",
)

agent_2 = ConversableAgent(
    name="Debate-Agent-B",
    llm_config=llm_config,
    system_message=agent_2_system_message,
    human_input_mode="NEVER",
)

agent_3 = ConversableAgent(
    name="Decision-Agent",
    llm_config=llm_config,
    system_message=agent_3_system_message,
    human_input_mode="NEVER",
)

group_chat = GroupChat(
    agents=[agent_1, agent_2, agent_3],
    messages=[],
    max_round=4,
    speaker_selection_method="round_robin"
)

group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config,
)

## Evaluation

In [167]:
def evaluate(question, response_A, response_B, label):

    message = f""" 
    Question: {question}

    Response A: {response_A}

    Response B: {response_B}
    """

    chat_results = agent_3.initiate_chat(
        group_chat_manager,
        message=message,
        summary_method="last_msg",
    )

    result_str = str(chat_results.chat_history[-1]["content"])
    print(result_str)

    pattern = r'"response"\s*:\s*"?([AB])"?'

    match = re.search(pattern, result_str)
    system_decision = match.group(1) if match else "X"
    print(system_decision)

    ground_truth = "A" if "A>" in label else "B"

    is_correct = system_decision == ground_truth

    return {
        "system_decision": system_decision,
        "ground_truth": ground_truth,
        "is_correct": is_correct
    }


In [168]:
num_rows = 100

df_final_subset = df_final.head(num_rows)

# normal position
results_1 = []

for _, row in tqdm(df_final_subset.iterrows(), total=num_rows, desc="Progress"):
    result = evaluate(
        question=row["question"],
        response_A=row["response_A"],
        response_B=row["response_B"],
        label=row["label"],
    )
    results_1.append(result)

results_1_df = pd.DataFrame(results_1)
results_1_df.to_csv('Results/competitive_1.csv', index=False)

accuracy_1 = results_1_df["is_correct"].mean()
print(f"Accuracy_1: {accuracy_1:.2%}")

position_distribution_1 = results_1_df["system_decision"].value_counts(normalize=True) * 100
print(f"Position_Distribution_1: {position_distribution_1}")

# swapped position
results_2 = []

for _, row in tqdm(df_final_subset.iterrows(), total=num_rows, desc="Progress"):
    result = evaluate(
        question=row["question"],
        response_A=row["response_B"], # positions swap 
        response_B=row["response_A"], # positions swap 
        label=row["label"],
    )

    # position swap back
    if result["system_decision"] == "A":
        result["system_decision"] = "B"
    elif result["system_decision"] == "B":
        result["system_decision"] = "A"

    if result["is_correct"] == True:
        result["is_correct"] = False
    elif result["is_correct"] == False:
        result["is_correct"] = True

    results_2.append(result)

results_2_df = pd.DataFrame(results_2)
results_2_df.to_csv('Results/competitive_2.csv', index=False)

accuracy_2 = results_2_df["is_correct"].mean()
print(f"Accuracy_2: {accuracy_2:.2%}")

position_distribution_2 = results_2_df["system_decision"].value_counts(normalize=True) * 100
print(f"Position_Distribution_2: {position_distribution_2}")



Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Decision-Agent (to chat_manager):

 
    Question: Under standard temperature and pressure conditions, compare the relative rates at which inert gases,Ar, He, and Kr diffuse through a common orifice.
(A) .1002 : .3002 : .4002
(B) .3582 : .4582 : .0582
(C) .2582 : .4998 : .3092
(D) .1582 : .6008 : .2092
(E) .1582 : .4998 : .1092
(F) .2002 : .4002 : .1092
(G) .1582 : .3998 : .2592
(H) .2502 : .4502 : .1502
(I) .2082 : .5998 : .1592
(J) .1802 : .4802 : .2802
If you cannot determine the correct multiple-choice answer, take your best guess. Once you have your answer, please duplicate that letter five times in a single string. For example, if the answer is K, then write KKKKK.
Let's think step by step.

    Response A: Let's approach this step-by-step:

1) The rate of diffusion of gases is inversely proportional to the square root of their molecular masses. This is known as Graham's Law.

2) We need to find the relative rates, so we'll use the formula: 
   Rate ∝ 1/√(Molecular Mass)

3) Mole

Progress:   1%|          | 1/100 [00:00<00:21,  4.66it/s]

After evaluating both responses, I conclude that Response A is the more factual correct response. 

Response A correctly applies Graham's Law of Diffusion and accurately calculates the normalized rates of diffusion for Argon (Ar), Helium (He), and Krypton (Kr). It arrives at the correct normalized values and identifies the closest match to the provided options as option D (0.1582 : 0.6008 : 0.2092). 

In contrast, Response B incorrectly claims that the closest match is option E (0.1582 : 0.4998 : 0.1092), which does not align with the calculated diffusion rates. This misidentification indicates a flaw in the reasoning presented in Response B.

Both responses follow a logical structure, but the accuracy of the calculations and the identification of the correct option in Response A make it the more factual correct response.

Therefore, the final conclusion is: {"response": A}
A
Decision-Agent (to chat_manager):

 
    Question: The total cost of producing x cameras is C(x) = 2 + x^3. Wha

Progress:   2%|▏         | 2/100 [00:00<00:16,  6.04it/s]

...

After evaluating both responses, I conclude that Response A is the more factual correct response. 

Response A effectively identifies apoptosis as a primary mechanism through which many viruses induce cell death. This aligns with established scientific understanding in virology, where apoptosis is recognized as a significant pathway for viral-induced cell death. The response systematically eliminates less common mechanisms and focuses on the most supported scientific evidence, which strengthens its argument.

In contrast, while Response B discusses the depletion of cellular resources as a consequence of viral replication, it does not adequately address that this is more of a secondary effect rather than a direct killing mechanism. The depletion of energy resources can lead to cell dysfunction but does not universally result in immediate cell death, making it less applicable as a primary mechanism across all viral types.

Therefore, based on the analysis and the current understanding of

Progress:  99%|█████████▉| 99/100 [00:15<00:00,  6.28it/s]

After evaluating the arguments presented by both Debate-Agent A and Debate-Agent B, I conclude that Response A is the more factual correct response.

Response A correctly identifies the pairs of numbers that meet the criteria of having a least common multiple (LCM) of 60 and a greatest common factor (GCF) of 2. It accurately narrows down the possible pairs and correctly concludes that 6 and 10 is the only valid pair that satisfies both conditions. 

On the other hand, Response B incorrectly states that the pair 10 and 12 meets the GCF requirement of 2, but it fails to recognize that the LCM of 10 and 12 is not 60, which is a critical error. The LCM of 10 and 12 is actually 60, but the GCF is not 2; it is 2, which does not invalidate the pair. However, the pair 6 and 10 is the only one that meets both the GCF and LCM conditions correctly.

Thus, the logical and systematic approach taken in Response A leads to the correct conclusion, while Response B's conclusion is based on a misunderst

Progress: 100%|██████████| 100/100 [00:16<00:00,  6.21it/s]

After evaluating both responses, I conclude that Response A is the more factual correct response. 

Response A provides a thorough analysis of each option, correctly identifying that neural networks are capable of handling non-linear relationships and various types of data. It also accurately points out that the only true statement is (H), while emphasizing that (C) "None of the above" is the most comprehensive answer given the number of false statements. This reflects a deeper understanding of the implications of the false statements collectively.

In contrast, Response B, while also identifying the inaccuracies, fails to recognize the broader context provided by (C) and incorrectly concludes that (H) is the best answer without acknowledging the significance of the other false statements. This oversight diminishes the factual accuracy of Response B.

Thus, the more factual correct response is Response A: {"response": A}
A
Accuracy_2: 50.00%
Position_Distribution_2: system_decision
B  